In [ ]:

!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2

!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

%tensorflow_version 2.x

!pip install tensorflow-gpu==2.0
import tensorflow as tf
!pip install pyarrow


In [ ]:
!apt install libomp-dev

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pwd; ls

In [ ]:
#!cp BioBert.tar.gz "drive/MyDrive/Colab Notebooks/"
#!cp DataAndCheckpoint.zip "drive/MyDrive/Colab Notebooks/"


In [ ]:
!pip install "drive/MyDrive/Colab Notebooks/docproduct.zip"

In [ ]:
#!pip cache purge

In [ ]:
!pip install "drive/MyDrive/Colab Notebooks/gpt2_estimator.zip"

In [ ]:
#!rm DataAndCheckpoint.zip

In [ ]:
#@title Downaload all model checkpoints, and question/answer data. 

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
import os
import requests

import urllib.request
import shutil

if not os.path.exists('BioBert.tar.gz'):  
  if os.path.exists('drive/MyDrive/Colab Notebooks/BioBert.tar.gz'):
    shutil.copy('drive/MyDrive/Colab Notebooks/BioBert.tar.gz','.')
  else:
    # Download the file from `url` and save it locally under `file_name`:
    urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_v1.0_pubmed_pmc.tar.gz', 'BioBert.tar.gz')
    !cp BioBert.tar.gz "drive/MyDrive/Colab Notebooks/"
  
if not os.path.exists('BioBertFolder'):
    os.makedirs('BioBertFolder')
    
import tarfile
tar = tarfile.open("BioBert.tar.gz")
tar.extractall(path='BioBertFolder/')
tar.close()

file_id = '1uCXv6mQkFfpw5txGnVCsl93Db7t5Z2mp'
download_file_from_google_drive(file_id, 'Float16EmbeddingsExpanded5-27-19.pkl')

if not os.path.exists('DataAndCheckpoint.zip'):
  if os.path.exists('drive/MyDrive/Colab Notebooks/DataAndCheckpoint.zip'):
    shutil.copy('drive/MyDrive/Colab Notebooks/DataAndCheckpoint.zip','.')
  else:
    file_id = 'https://onedrive.live.com/download?cid=9DEDF3C1E2D7E77F&resid=9DEDF3C1E2D7E77F%2132792&authkey=AEQ8GtkcDbe3K98'
    urllib.request.urlretrieve( file_id, 'DataAndCheckpoint.zip')
    !cp DataAndCheckpoint.zip "drive/MyDrive/Colab Notebooks/"


if not os.path.exists('newFolder'):
    os.makedirs('newFolder')

import zipfile
zip_ref = zipfile.ZipFile('DataAndCheckpoint.zip', 'r')
zip_ref.extractall('newFolder')
zip_ref.close()

In [ ]:
#@title Load model weights and Q&A data. Double click to see code

from docproduct.predictor import RetreiveQADoc

pretrained_path = 'BioBertFolder/biobert_v1.0_pubmed_pmc/'
# ffn_weight_file = None
bert_ffn_weight_file = 'newFolder/models/bertffn_crossentropy/bertffn'
embedding_file = 'Float16EmbeddingsExpanded5-27-19.pkl'

doc = RetreiveQADoc(pretrained_path=pretrained_path,
ffn_weight_file=None,
bert_ffn_weight_file=bert_ffn_weight_file,
embedding_file=embedding_file)

In [ ]:
while True:
  print('Enter Question: ')
  question_text = input()
  if len(question_text) <= 1:
    print('Quitting.. Bye.')
    break 
  search_similarity_by = 'answer'  #@param ['answer', "question"]
  number_results_toReturn=1 #@param {type:"number"}
  answer_only=True #@param ["False", "True"] {type:"raw"}
  returned_results = doc.predict( question_text ,
                  search_by=search_similarity_by, topk=number_results_toReturn, answer_only=answer_only)
  print('')
  for jk in range(len(returned_results)):
      print("Result ", jk+1)
      print(returned_results[jk])
      print('')

In [ ]:
!pip install flask_ngrok

In [ ]:
# !pwd; ls; chmod +777 ngrok

In [ ]:
# !./ngrok authtoken 23vZ4puLfclclQ5f5wSzfJ4c0eS_65exTsXLh7GaYWDQGuktV

In [ ]:
!npm install -g localtunnel


In [ ]:
!nohup lt --port 5000 -s aidoc &

In [ ]:
!ps aux

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask 
from flask import request
app = Flask(__name__)

@app.route("/")
def home():
    return "<h1>This is medical assistant, please tell your symptoms..</h1>"

@app.route("/medicaladvice", methods=['GET', 'POST'])
def medicaladvice():
  question = request.args.get("question")
  print('Question: ', question)
  if question is None and request.content_type == 'application/json':
    question = request.json.get("question")
    if question is None:
      return 'Empty question!!'
    print('Question from json data:', question)
  elif not isinstance(question, str):
      return 'Could not find question in request'

  search_similarity_by = 'answer'  #@param ['answer', "question"]
  number_results_toReturn = 1
  answer_only=True #@param ["False", "True"] {type:"raw"}
  returned_results = doc.predict(question ,
                  search_by=search_similarity_by, 
                  topk=number_results_toReturn, 
                  answer_only=answer_only)
  print ("Answer: ", returned_results[0])
  return returned_results[0]

app.run()